In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import scipy.stats as stats
from scipy.stats import pearsonr
from tqdm import tqdm
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

In [21]:
df = pd.read_parquet('../data/dataframe_compressed.parquet')

In [2]:
df_model = pd.read_parquet('../data/df_cleaned_for_classification_models.parquet')

In [32]:
df_old = pd.read_csv('../data/df_clean_before_NaNs_with_datetime_infos_with_geo.csv', index_col = 'IncidentNumber', dtype={'IncidentNumber': 'str'}, parse_dates=['DateTimeCall'])

In [50]:
log_threshold = np.log(360)
df_model['ResponseTimeBinary'] = (df_model['TotalResponseTimeLog'] <= log_threshold).astype(int)

In [3]:
df_model

,IncidentGroup_Fire,IncidentGroup_Special Service,AggregatedPropertyCategory_Outdoor,AggregatedPropertyCategory_Residential,AggregatedPropertyCategory_Vehicle,CellEastingNorthing2500_502500-175000,CellEastingNorthing2500_502500-177500,CellEastingNorthing2500_502500-180000,CellEastingNorthing2500_502500-182500,CellEastingNorthing2500_502500-187500,...,IsBankholiday,IsWeekend,DistanceStationLog,Hour_sin,Hour_cos,Weekday_sin,Weekday_cos,Month_sin,Month_cos,ResponseTimeBinary
IncidentNumber,,,,,,,,,,,,,,,,,,,,,
235138081,0,1,0,0,1,0,0,0,0,0,...,1,0,7.207600,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025,1
2091,1,0,1,0,0,0,0,0,0,0,...,1,0,6.454777,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025,1
3091,1,0,1,0,0,0,0,0,0,0,...,1,0,5.940170,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025,1
5091,1,0,1,0,0,0,0,0,0,0,...,1,0,7.031189,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025,1
6091,0,0,0,1,0,0,0,0,0,0,...,1,0,6.281069,0.000000,1.000000,0.433884,-0.900969,0.500000,0.866025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
067897-30042024,0,1,0,1,0,0,0,0,0,0,...,0,0,6.748761,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000,1
067898-30042024,0,0,0,1,0,0,0,0,0,0,...,0,0,7.248166,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000,1
067896-30042024,0,0,0,1,0,0,0,0,0,0,...,0,0,7.576672,-0.258819,0.965926,0.000000,1.000000,0.866025,-0.500000,0


In [44]:
#Converting Hours to cyclic
p = 24
df_cyc.loc[:,'Hour_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'Hour'] / p)
df_cyc.loc[:,'hour_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'Hour'] / p)

#Converting Weekdays to cyclic
p = 7
df_cyc.loc[:,'Weekday_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)
df_cyc.loc[:,'Weekday_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)

#Converting Months to cyclic
p = 12
df_cyc.loc[:,'Month_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'Month'] / p)
df_cyc.loc[:,'Month_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'Month'] / p)

C:\Users\Isi\AppData\Local\Temp\ipykernel_9768\313601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cyc.loc[:,'Weekday_sin'] = np.sin(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)
C:\Users\Isi\AppData\Local\Temp\ipykernel_9768\313601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cyc.loc[:,'Weekday_cos'] = np.cos(2 * np.pi * df_cyc.loc[:,'WeekDay'] / p)


In [18]:
df_model['DistanceStationLog'] = np.log(df_model['DistanceStation'])
df_model['TotalResponseTimeLog'] = np.log(df_model['TotalResponseTime'])
df_model = df_model.drop(['TotalResponseTime', 'DistanceStation'], axis = 1)

In [5]:
X = df_model.drop(['ResponseTimeBinary'], axis = 1)
y = df_model['ResponseTimeBinary']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [70]:
# saving the datasets to files
df_model.to_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/df_cleaned_for_classification_models.parquet')
X_train.to_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/X_train_classification.parquet')
X_test.to_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/X_test_classification.parquet')

In [14]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Load your data (assuming it is a DataFrame called 'df')
# df = pd.read_csv('path_to_your_file.csv')  # Replace with your actual data loading step

# Add a constant to the dataframe for the intercept
X = add_constant(df_model)  # Replace 'df' with your DataFrame variable

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['TotalResponseTimeLog'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print(vif_data)

                       TotalResponseTimeLog           VIF
0                                     const  64435.600245
1                        IncidentGroup_Fire      1.556885
2             IncidentGroup_Special Service      1.306543
3        AggregatedPropertyCategory_Outdoor      1.955774
4    AggregatedPropertyCategory_Residential      1.821262
..                                      ...           ...
316                                Hour_cos      1.027838
317                             Weekday_sin      2.829144
318                             Weekday_cos      1.118115
319                               Month_sin      1.006214
320                               Month_cos      1.011094

[321 rows x 2 columns]


In [59]:
vif_data.sort_values(by='VIF', ascending=False).head(10)

,TotalResponseTimeLog,VIF
0,const,64435.600245
138,CellEastingNorthing2500_527500-180000,2229.681818
157,CellEastingNorthing2500_530000-180000,1560.657411
176,CellEastingNorthing2500_532500-180000,1305.187360
119,CellEastingNorthing2500_525000-180000,1122.932869
177,CellEastingNorthing2500_532500-182500,1079.925283
137,CellEastingNorthing2500_527500-177500,1050.291067
118,CellEastingNorthing2500_525000-177500,1025.495914
158,CellEastingNorthing2500_530000-182500,935.417557
156,CellEastingNorthing2500_530000-177500,914.511247


In [38]:
model = LinearRegression()

In [39]:
model.fit(X_train, y_train)

LinearRegression()

In [40]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [41]:
print(f"Mean squared error: {mse}")
print(f"Mean absolute error: {mean_absolute_error(y_test, y_pred)}")
print(f"Root Mean squared error: {np.sqrt(mse)}")
print(f"R2 score: {r2}")
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)
print("Cross-validated MSE:", -scores.mean())

Mean squared error: 14884.907485424852
Mean absolute error: 83.75868634094209
Root Mean squared error: 122.00371914587215
R2 score: 0.1944172281110158
Cross-validated MSE: 14900.788502761387


In [75]:
#print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)

Model Intercept: 3.9949890287834595


In [44]:
df_model['TotalResponseTime'].mean()

321.3716625566429

In [63]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

print("Logistic Regression:")
print(classification_report(y_test, y_pred_log_reg))
print("Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("ROC AUC:", roc_auc_score(y_test, y_pred_log_reg))

Logistic Regression:
              precision    recall  f1-score   support

           0       0.66      0.29      0.40     90243
           1       0.76      0.94      0.84    217298

    accuracy                           0.75    307541
   macro avg       0.71      0.61      0.62    307541
weighted avg       0.73      0.75      0.71    307541

Accuracy: 0.7472792245586767
ROC AUC: 0.6140058862580664


In [66]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_smote, y_train_smote)

y_pred = log_reg.predict(X_test)

print("Logistic Regression with SMOTE:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred))

C:\Users\Isi\anaconda3\envs\FireBrigade\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression with SMOTE:
              precision    recall  f1-score   support

           0       0.47      0.63      0.54     90243
           1       0.82      0.71      0.76    217298

    accuracy                           0.69    307541
   macro avg       0.65      0.67      0.65    307541
weighted avg       0.72      0.69      0.70    307541

Accuracy: 0.685989835501608
ROC AUC: 0.669160780422318


In [68]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

log_reg = LogisticRegression(max_iter=1000)
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train_smote, y_train_smote)

best_log_reg = grid_search.best_estimator_
y_pred = best_log_reg.predict(X_test)

print("Best Logistic Regression with SMOTE:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred))

C:\Users\Isi\anaconda3\envs\FireBrigade\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Isi\anaconda3\envs\FireBrigade\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

Best Logistic Regression with SMOTE:
              precision    recall  f1-score   support

           0       0.47      0.63      0.54     90243
           1       0.82      0.71      0.76    217298

    accuracy                           0.69    307541
   macro avg       0.65      0.67      0.65    307541
weighted avg       0.72      0.69      0.70    307541

Accuracy: 0.6858825327354726
ROC AUC: 0.6691075250905887


In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
pca = PCA(n_components=15)  # Reduce to 2 dimensions for simplicity
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [18]:
knn = KNeighborsClassifier(n_neighbors=12)
knn.fit(X_train_pca, y_train)

KNeighborsClassifier(n_neighbors=12)

In [19]:
y_pred = []
for i in tqdm(range(len(X_test_pca)), desc="Predicting"):
    y_pred.append(knn.predict([X_test_pca[i]]))
y_pred = np.array(y_pred).flatten()

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Predicting: 100%|██████████| 307541/307541 [09:00<00:00, 568.70it/s]


Accuracy: 0.73
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.40      0.47     90243
           1       0.78      0.87      0.82    217298

    accuracy                           0.73    307541
   macro avg       0.67      0.63      0.64    307541
weighted avg       0.71      0.73      0.72    307541

Confusion Matrix:
[[ 35739  54504]
 [ 27431 189867]]


In [20]:
y_pred = []
for i in tqdm(range(len(X_train_pca)), desc="Predicting"):
    y_pred.append(knn.predict([X_train_pca[i]]))
y_pred = np.array(y_pred).flatten()

accuracy = accuracy_score(y_train, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_train, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_train, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred))

Predicting: 100%|██████████| 1230163/1230163 [42:06<00:00, 486.98it/s] 


Accuracy: 0.77
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.47      0.55    361454
           1       0.80      0.90      0.85    868709

    accuracy                           0.77   1230163
   macro avg       0.73      0.68      0.70   1230163
weighted avg       0.76      0.77      0.76   1230163

Confusion Matrix:
[[168622 192832]
 [ 88606 780103]]
